# Crypto 2 - OTP 2

> Okay it should be complete now
> 
> - mechfrog88
>
> Files: [main.py](main.py)

This is OTP but with less cheese. The setup is as follows:
- You are given a random 1024-bit prime $p$, and $g = 35^{10}$.
- The OTP consists of six 16-bit values: $o_1, o_2, \ldots, o_6$.
- There are six private values (less than $p$): $x_1, x_2, \ldots, x_6$.
- There is another private value $r$, also less than $p$.
- We are given the public versions of the private values: $g^{x_i} \pmod{p}$.
- We are also given the value of $g^{r} \pmod{p}$ and $g^{r x_i + o_i} \pmod{p}$.
- We wish to recover the OTP ($o_i$).

Actually we also have the dot product of `OTP` and `token` as in part 1, but I've ignored it as we will actually solve without it.

So as always with discrete logs, it is generally intractable to fully solve for the exponent in $GF(p)$, unless $p-1$ happens to be smooth, which is very rare. But it doesn't need to be smooth since we don't need to full exponent -- recall that our aim is to learn $o_i$ which are 16-bit values. This means that if we can solve discrete logs mod any subgroup of order more than 65536, we are done!

And this is exactly what happened.
1. Keep generating remote instances until $p-1$ has a prime factor $q$ between 17 and 24 bits long. Technically it doesn't need to be prime, but this saves us from worrying about whether $g$ is in fact a generator.
2. Work entirely in $\bmod{q}$.
    - Solve for $r \bmod{q}$.
    - Solve for $o_i \bmod{q}$.
    
And that's about it really. So here's the solve script.

In [1]:
from sage.all import *
from sage.rings.factorint import factor_trial_division
from pwn import *

sh = process(['python', 'main.py'])
#sh = remote('34.124.157.94', 19622)
p = int(sh.readline()[3:])
F = GF(p, proof=False)
pub = [F(x) for x in eval(sh.readline()[5:])]
sh.sendline(b'10000000000,10000000000,10000000000,10000000000,10000000000,10000000000')
sh.readuntil(b'OTP Hash: ')
oh = [F(x) for x in eval(sh.readline())]

q = next(a for a,_ in factor_trial_division(p-1, 2**24)[:-1] if a>65536)
print(q, q.nbits())

g = F(35**10)

def dlog(target):
    return discrete_log(target ** (p // q), g ** (p // q), q)

r = dlog(oh[0])
    
soln = [dlog(oh[i+1] / pub[i]**r) for i in range(6)]
sh.sendline(','.join(map(str,soln)).encode())
sh.readall()

[x] Starting local process '/usr/bin/python'
[+] Starting local process '/usr/bin/python': pid 12172
5767847 23
[x] Receiving all data
[x] Receiving all data: 69B
[x] Receiving all data: 95B
[*] Process '/usr/bin/python' stopped with exit code 0 (pid 12172)
[+] Receiving all data: Done (95B)


b'Sending OTP....\nNow insert your otp (6 integers separated by comma):\nWelcome Admin! <REDACTED>\n'

Actually the flag got changed at some point because the original challenge had a bug where you multiplied `pow(a,b,p)` with `pow(c,d,p)` but did not reduce the product mod p, so you could essentially just check if it was a factor.

Anyway, the new flag is `grey{I_forgot_to_remove_the_debug_message_qgn9fe58yMwJdMRf}`.